# Explore IBM Q
Exploring the backend data visually on IBM Q Experience is a handy but strictly manual process. Sometimes you might want to include backend information when running your programs, for example to select the appropriate backend or dynamically apply your gates on the “best” qubits. To do this you need to dig into the available backend information by using Qiskit.

Let's start by importing the IBMQ method and load our account.

In [1]:
from qiskit import IBMQ

#Save and store API key locally.
#IBMQ.save_account('MY_API_TOKEN')

IBMQ.load_account()
provider = IBMQ.get_provider()

Then, let's pick the least busy available backend for our exercise

In [2]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(provider.backends(operational=True, simulator=False))
#backend = provider.get_backend('ibmq_vigo')
print("Selected backend:",backend.status().backend_name)


Selected backend: ibmq_vigo


Let's start by taking a look at the backend configuration details.

In [3]:
backend.configuration()

QasmBackendConfiguration(allow_object_storage=True, allow_q_circuit=False, allow_q_object=True, backend_name='ibmq_vigo', backend_version='1.0.1', basis_gates=['u1', 'u2', 'u3', 'cx', 'id'], conditional=False, coupling_map=[[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]], credits_required=True, description='5 qubit device Vigo', gates=[GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='id', parameters=[], qasm_def='gate id q { U(0,0,0) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='u1', parameters=['lambda'], qasm_def='gate u1(lambda) q { U(0,0,lambda) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='u2', parameters=['phi', 'lambda'], qasm_def='gate u2(phi,lambda) q { U(pi/2,phi,lambda) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='u3', parameters=['theta', 'phi', 'lambda'], qasm_def='gate u3(theta,phi,lambda) q { U(theta,phi,lambda) q; }'), GateConfig(coupling_map=[[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1]

We can for example take a look at the number of qubits for the backend by getting the `n_qubits` parameter from the backend configuration.

In [4]:
print("Number of qubits:", backend.configuration().n_qubits)

Number of qubits: 5


# Understanding the qubit coupling map for the chip

**Time for you to do some work!** 

Another aspect of the backend that is important for entangling qubits is the interconnectivity between the individual qubits. Not all qubits can communicate directly with all other qubits. The coupling map illustrates which qubits can communicate directly with which, from the first qubit to the second. 

Use the `coupling_map` parameter from the backend configuration to see this information.

In [5]:
# Your code here...

backend.configuration().coupling_map

[[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]]

# More backend information
Other interesting information is the number of maximum shots you can run your quantum program and the max number of experiments you can run on the backend per day. It might also be interesting to know the number of pending jobs for the selected backend. For this you can use `backend.status().pending_jobs` which also includes other status information such as if the backend is active and which backend version it runs.. 

In [8]:
print("Max shots:", backend.configuration().max_shots)
print("Max experiments:", backend.configuration().max_experiments)
print("Pending jobs:", backend.status().pending_jobs)

Max shots: 8192
Max experiments: 75
Pending jobs: 1


You can also use some simple Python scripting to compare the available backends based on different criteria.

In [ ]:
# Get all available and operational backends.
available_backends = provider.backends(operational=True)

# Fish out criteria to compare
print("{0:20} {1:<10} {2:<10} {3:<10}".format("Name","#Qubits","Max exp.","Pending jobs"))
print("{0:20} {1:<10} {2:<10} {3:<10}".format("++++","+++++++","++++++++","++++++++++++"))

for n in range(0, len(available_backends)):
    available_backend = provider.get_backend(str(available_backends[n]))
    print("{0:20} {1:<10} {2:<10} {3:<10}"
          .format(available_backend.name(), available_backend.configuration().n_qubits,
                                  available_backend.configuration().max_experiments,
                                  available_backend.status().pending_jobs))
    
    

# The physical properties of a backend
We can now take a look at the physical properties of the selected backend. Let's start by collecting everything in one big pile.

In [9]:
backend.properties()


BackendProperties(backend_name='ibmq_vigo', backend_version='1.0.1', gates=[Gate(gate='id', name='id_0', parameters=[Nduv(date=datetime.datetime(2019, 10, 18, 6, 27, 25, tzinfo=tzutc()), name='gate_error', unit='', value=0.0003124880456005525), Nduv(date=datetime.datetime(2019, 10, 20, 6, 22, 19, tzinfo=tzutc()), name='gate_length', unit='ns', value=35.55555555555556)], qubits=[0]), Gate(gate='u1', name='u1_0', parameters=[Nduv(date=datetime.datetime(2019, 10, 18, 6, 27, 25, tzinfo=tzutc()), name='gate_error', unit='', value=0.0), Nduv(date=datetime.datetime(2019, 10, 20, 6, 22, 19, tzinfo=tzutc()), name='gate_length', unit='ns', value=0.0)], qubits=[0]), Gate(gate='u2', name='u2_0', parameters=[Nduv(date=datetime.datetime(2019, 10, 18, 6, 27, 25, tzinfo=tzutc()), name='gate_error', unit='', value=0.0003124880456005525), Nduv(date=datetime.datetime(2019, 10, 20, 6, 22, 19, tzinfo=tzutc()), name='gate_length', unit='ns', value=35.55555555555556)], qubits=[0]), Gate(gate='u3', name='u3_0

From this mess, which contains all relevant hardware information for the backend, we can dig out the meat. Let us start with some simple data such as the backend name.

In [10]:
backend.properties().backend_name


'ibmq_vigo'

Other properties involve more complex information, such as the performance information for the qubits. Let us take a look at the available qubit information for this backend, which is returned as a Python list.

In [11]:
backend.properties().qubits

[[Nduv(date=datetime.datetime(2019, 10, 20, 6, 6, 28, tzinfo=tzutc()), name='T1', unit='µs', value=120.80666407747465),
  Nduv(date=datetime.datetime(2019, 8, 9, 16, 59, 15, tzinfo=tzutc()), name='T2', unit='µs', value=18.38638524492989),
  Nduv(date=datetime.datetime(2019, 10, 20, 6, 22, 19, tzinfo=tzutc()), name='frequency', unit='GHz', value=4.795982361476481),
  Nduv(date=datetime.datetime(2019, 10, 20, 6, 6, 15, tzinfo=tzutc()), name='readout_error', unit='', value=0.07699999999999996),
  Nduv(date=datetime.datetime(2019, 10, 20, 6, 6, 15, tzinfo=tzutc()), name='prob_meas0_prep1', unit='', value=0.058),
  Nduv(date=datetime.datetime(2019, 10, 20, 6, 6, 15, tzinfo=tzutc()), name='prob_meas1_prep0', unit='', value=0.09599999999999997)],
 [Nduv(date=datetime.datetime(2019, 10, 20, 6, 6, 28, tzinfo=tzutc()), name='T1', unit='µs', value=98.94382217473189),
  Nduv(date=datetime.datetime(2019, 10, 20, 6, 8, 33, tzinfo=tzutc()), name='T2', unit='µs', value=100.0717894424845),
  Nduv(date=

Again, let us dig in an and take a look at some of the available qubit data, such as the T1 and T2 decoherence values, the frequency, and the readout error for the qubits. For this exercise we can write a simple Python `for` loop that prints the `name`, `value`, and `unit` for the relevant data entry for each qubit. 

In [12]:
for q in range (0, backend.configuration().n_qubits):
    print("Qubit",q,":")
    for n in range (0, len(backend.properties().qubits[0])):
        print(backend.properties().qubits[q][n].name,
              "=",
              backend.properties().qubits[q][n].value,backend.properties().qubits[q][n].unit)

Qubit 0 :
T1 = 120.80666407747465 µs
T2 = 18.38638524492989 µs
frequency = 4.795982361476481 GHz
readout_error = 0.07699999999999996 
prob_meas0_prep1 = 0.058 
prob_meas1_prep0 = 0.09599999999999997 
Qubit 1 :
T1 = 98.94382217473189 µs
T2 = 100.0717894424845 µs
frequency = 4.940047536530192 GHz
readout_error = 0.015000000000000013 
prob_meas0_prep1 = 0.026000000000000023 
prob_meas1_prep0 = 0.004 
Qubit 2 :
T1 = 96.2923872661221 µs
T2 = 167.79834206764465 µs
frequency = 4.833251787417111 GHz
readout_error = 0.016000000000000014 
prob_meas0_prep1 = 0.02200000000000002 
prob_meas1_prep0 = 0.01 
Qubit 3 :
T1 = 83.71085430465864 µs
T2 = 92.73468776860886 µs
frequency = 4.807990609310798 GHz
readout_error = 0.010000000000000009 
prob_meas0_prep1 = 0.016000000000000014 
prob_meas1_prep0 = 0.004 
Qubit 4 :
T1 = 96.95474132935135 µs
T2 = 45.80803996746207 µs
frequency = 4.74989464831662 GHz
readout_error = 0.010000000000000009 
prob_meas0_prep1 = 0.01 
prob_meas1_prep0 = 0.010000000000000009 


In addition to the Qiskit way of pulling out the data you can also use the graphical Jupyter view of this information. 

In [23]:
%matplotlib inline
from qiskit.tools.jupyter import *
%qiskit_backend_overview
%qiskit_backend_monitor backend


UsageError: Line magic function `%qiskit_backend_monitor` not found.


Depending on the type of quantum program that you are writing, certain aspects of the backend might be important to your, and you might want to include these directly when you code your program. For example, you might be interested in the qubits with the smallest gate errors and readout errors, or if you are running deep circuits you might be interested in long T1 and T2 times. Another important piece of information might be the quality of the CNOT gates supported by the chip. 

For more assistance on the backend, use `help(backend)`

In [20]:
help(backend)

Help on IBMQBackend in module qiskit.providers.ibmq.ibmqbackend object:

class IBMQBackend(qiskit.providers.basebackend.BaseBackend)
 |  IBMQBackend(configuration, provider, credentials, api)
 |  
 |  Backend class interfacing with an IBMQ backend.
 |  
 |  Method resolution order:
 |      IBMQBackend
 |      qiskit.providers.basebackend.BaseBackend
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, configuration, provider, credentials, api)
 |      Initialize remote backend for IBM Quantum Experience.
 |      
 |      Args:
 |          configuration (BackendConfiguration): configuration of backend.
 |          provider (IBMQProvider): provider.
 |          credentials (Credentials): credentials.
 |          api (IBMQConnector):
 |              api for communicating with the Quantum Experience.
 |  
 |  __repr__(self)
 |      Official string representation of a Backend.
 |      
 |      Note that, by Qiskit convention, it is consciously *not*